In [4]:
import anndata
import scvi
import numpy as np
import os
import scvi
import scanpy as sc

#path to eval dataset
lung_eval_path = "Lung_eval.h5ad"
#path to reference
tabula_sapiens_filepath = 'Lung_ref.h5ad'
#path to user data
input_anndata = 'Lung_test.h5ad'


tissue = 'Lung'
output_folder_name = 'lung_evaluation'
use_gpu = True
use_10X_only= True
batch_correction_conditions = ['method']
annotation_key = 'manual_annotation'



#scVI arguments:
scvi_model = None
scanvi_model= None
n_scvi_epochs= 400
n_scanvi_epochs = 15

output_folder = os.path.join('data', output_folder_name)


In [5]:
a = anndata.read(lung_eval_path)

tsp_idx = (a.obs['donor'] == 'TSP1') | (a.obs['donor'] == 'TSP2')
test = a[tsp_idx].copy()
test.X = np.around(np.expm1(test.X))
test.write('Lung_test.h5ad')

ref = a[~tsp_idx].copy()
ref.obs['tissue'] = 'Lung'
ref.write('Lung_ref.h5ad')

/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
... storing 'tissue' as categorical


In [6]:
def setup_dataset(input_anndata, ts_filepath, tissue = None, use_10X_only = False, batch_correction_conditions = None):
    #TODO check that the input vars are the same
    train_data = sc.read_h5ad(ts_filepath)
    test_data = sc.read_h5ad(input_anndata)
    if tissue is not None:
        train_data = train_data[train_data.obs['tissue'] == tissue]
    if use_10X_only:
        train_data = train_data[train_data.obs['method'] == "10X"]
    if batch_correction_conditions is None:
        full = train_data.concatenate(test_data)
        full.obs['_batch_indices'] = np.zeros(full.n_obs).astype(int)    
    else:
        condition = batch_correction_conditions[0]
        train_data.obs['_batch'] = train_data.obs[condition]

        if len(batch_correction_conditions) >= 2:
            for condition in batch_correction_conditions[1:]:
                x1 = train_data.obs['_batch'].to_numpy().astype(str) 
                underscore = ['_'] * train_data.n_obs
                x1 = np.char.add(x1, underscore)
                x2 = train_data.obs[condition].to_numpy().astype(str)
                train_data.obs['_batch']= np.char.add(x1, x2)
        batch_key = '_batch'

        all_batches = np.unique(train_data.obs['_batch'].astype(str))
        batch_indices = np.zeros(train_data.n_obs).astype(int)
        for i, batch in enumerate(all_batches):
            idx = np.where(train_data.obs[batch_key] == batch)
            batch_indices[idx] = i

        train_data.obs['_batch_indices'] = batch_indices
        test_data.obs['_batch_indices'] = np.zeros(test_data.n_obs).astype(int) + len(all_batches)
        full = train_data.concatenate(test_data)
    
    full.obs['_dataset'] = ['reference'] * train_data.n_obs + ['user'] * test_data.n_obs
    return full

In [7]:
full = setup_dataset(input_anndata, tabula_sapiens_filepath, tissue, use_10X_only, batch_correction_conditions)
sc.pp.highly_variable_genes(full, flavor='seurat_v3', subset=True, n_top_genes=4000)


Trying to set attribute `.obs` of view, copying.
/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True
/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [62]:
train_data = full[full.obs['_dataset'] == 'reference'].copy()
test_data = full[full.obs['_dataset'] == 'user'].copy()

In [63]:
import numpy as np

def subsample_dataset(train_data, labels_key, n_samples):
    sample_idx = []
    labels, counts = np.unique(train_data.obs[labels_key], return_counts=True)
    for i, label in enumerate(labels):
        label_locs = np.where(train_data.obs[labels_key] == label)[0]        
        if counts[i] < n_samples:
            sample_idx.append(label_locs)
        else:
            label_subset = np.random.choice(label_locs, n_samples, replace = False)
            sample_idx.append(label_subset)
    sample_idx = np.concatenate(sample_idx)
    return sample_idx

In [64]:
sample_idx = subsample_dataset(train_data, labels_key = 'manual_annotation', n_samples = 100)

In [65]:
train_data.obs['scanvi_labels'] = 'unknown'
train_data.obs['scanvi_labels'][sample_idx] = train_data.obs['manual_annotation'][sample_idx]

/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [54]:
test_data = test_data[test_data.obs['method'] == '10X'].copy()

In [61]:
scvi.data.setup_anndata(train_data, batch_key = 'method', labels_key = 'scanvi_labels')

INFO      Using batches from adata.obs["method"]                                             


AssertionError: scanvi_labels is not a valid key for in adata.obs

In [60]:
model = scvi.model.SCANVI(train_data,
                          unlabeled_category = 'unknown', 
                          use_cuda = True, 
                          n_layers=3, 
                          n_latent=50, 
                          dispersion='gene-batch')

ValueError: Please setup your AnnData with scvi.data.setup_anndata(adata) first

In [50]:
unsupervised_trainer_kwargs = dict(data_loader_kwargs=dict(pin_memory=True))
semisupervised_trainer_kwargs = dict(data_loader_kwargs=dict(pin_memory=True))
semisupervised_train_kwargs = dict(batch_size = 1024)
unsupervised_train_kwargs = dict(batch_size = 1024)

In [59]:
model.train(n_epochs_unsupervised = 1, n_epochs_semisupervised = 1, train_size=1.0,n_epochs_kl_warmup=10,unsupervised_trainer_kwargs=unsupervised_trainer_kwargs ,
            semisupervised_trainer_kwargs = semisupervised_trainer_kwargs,
            semisupervised_train_kwargs=semisupervised_train_kwargs,
            unsupervised_train_kwargs = unsupervised_train_kwargs)


INFO      Training Unsupervised Trainer for 1 epochs.                                        
INFO      Training SemiSupervised Trainer for 1 epochs.                                      
INFO      KL warmup phase exceeds overall training phaseIf your applications rely on the     
          posterior quality, consider training for more epochs or reducing the kl warmup.    
INFO      KL warmup for 10 epochs                                                            
Training...:   0%|          | 0/1 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [56]:
test_data.obs['scanvi_labels'] = 'unknown'

In [25]:
scvi.data.setup_anndata(test_data, batch_key = 'method', labels_key = 'scanvi_labels')

INFO      Using batches from adata.obs["method"]                                             
INFO      Using labels from adata.obs["scanvi_labels"]                                       
INFO      Using data from adata.X                                                            
INFO      Computing library size prior per batch                                             
INFO      Successfully registered anndata object containing 31117 cells, 4000 vars, 1        
          batches, 1 labels, and 0 proteins. Also registered 0 extra categorical covariates  
          and 0 extra continuous covariates.                                                 
INFO      Please do not further modify adata until model is trained.                         


/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [57]:
model.predict(test_data)

KeyError: 'data_registry'

In [19]:
adata = test_data
scdl = model._make_scvi_dl(adata=adata, indices=np.arange(adata.n_obs), batch_size = 1024)
_, pred = scdl.sequential().compute_predictions()


In [48]:
predictions = []
for p in pred:
    predictions.append(model._code_to_label[p])
pred = np.array(predictions)

KeyError: 'capillary endothelial cell'

In [22]:
import pandas as pd
pd.DataFrame(index = adata.obs_names, data =pred).to_csv('scanvi_pred.csv')

In [47]:
scvi.data.transfer_anndata_setup(train_data, test_data)

INFO      Using data from adata.X                                                            
INFO      Computing library size prior per batch                                             
INFO      Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']    
INFO      Successfully registered anndata object containing 31117 cells, 4000 vars, 1        
          batches, 43 labels, and 0 proteins. Also registered 0 extra categorical covariates 
          and 0 extra continuous covariates.                                                 


/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [66]:
a = scvi.data.synthetic_iid()
m = scvi.model.SCANVI(a, 'batch_1')

INFO      Using batches from adata.obs["batch"]                                              
INFO      Using labels from adata.obs["labels"]                                              
INFO      Using data from adata.X                                                            
INFO      Computing library size prior per batch                                             
INFO      Using protein expression from adata.obsm['protein_expression']                     
INFO      Using protein names from adata.uns['protein_names']                                
INFO      Successfully registered anndata object containing 400 cells, 100 vars, 2 batches, 3
          labels, and 100 proteins. Also registered 0 extra categorical covariates and 0     
          extra continuous covariates.                                                       
INFO      Please do not further modify adata until model is trained.                         


/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/envs/tabula_sapiens/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [68]:
m.train(1, 1)

INFO      Training Unsupervised Trainer for 1 epochs.                                        
INFO      Training SemiSupervised Trainer for 1 epochs.                                      
INFO      KL warmup phase exceeds overall training phaseIf your applications rely on the     
          posterior quality, consider training for more epochs or reducing the kl warmup.    
INFO      KL warmup for 400 epochs                                                           
Training...: 100%|██████████| 1/1 [00:00<00:00, 10.94it/s]
INFO      Training is still in warming up phase. If your applications rely on the posterior  
          quality, consider training for more epochs or reducing the kl warmup.              
INFO      Training time:  0 s. / 1 epochs                                                    
INFO      KL warmup phase exceeds overall training phaseIf your applications rely on the     
          posterior quality, consider training for more epochs or reducing the kl warmup.    
I